In [1]:
from pysgx import info, stocks, util
import importlib
import pandas as pd
import numpy as np

In [70]:
# Re-run this cell if pysgx module have hot changes
importlib.reload(info);
importlib.reload(stocks);
importlib.reload(util);

# Portfolio

In [2]:
# Load ALL stocks!
all_stocks = stocks.loads(info.all_tickers)

def get_portfolio(tickers):
    portfolio = []
    for stock in all_stocks:
        if stock.ticker in tickers:
            portfolio.append(stock)
    return portfolio

def get_overall_return(ByAdjClose, ByAddBackClose):
    df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
    returns = df['ByAdjClose'].dropna() + 1
    print(returns.product())

    df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
    returns = df['ByAddBackClose'] + 1
    print(returns.product())

# Rolling Test using AddBack Close Return

In [9]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAddBackClose):
                returns[stock.ticker] = r.ByAddBackClose

    # sort to find top 10 return stocks using AddBack Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)

    r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
    print("backtest 1Q", period_end, eval_period_end, r)
    ByAdjClose.append(r.ByAdjClose)
    ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

Evaluate 1Y since:  2000-01-01 dict_keys(['L02', '5AB', 'J36', 'CTO', 'H78', 'BTJ', 'BDA', 'LJ3', 'KUH', 'H02'])
backtest 1Q 2001-01-01 2001-04-01 ByAdjClose: -0.153297 ByAddBackClose: -0.152858

Evaluate 1Y since:  2000-04-01 dict_keys(['L02', 'LJ3', 'J36', 'H78', 'H02', 'C52', 'U14', '5AB', 'S63', 'S20'])
backtest 1Q 2001-04-01 2001-07-01 ByAdjClose:  0.046727 ByAddBackClose:  0.044415

Evaluate 1Y since:  2000-07-01 dict_keys(['600', 'T14', 'AZR', 'L02', 'LJ3', 'G07', 'H02', 'F13', 'WPC', 'J36'])
backtest 1Q 2001-07-01 2001-10-01 ByAdjClose: nan ByAddBackClose: -0.088402

Evaluate 1Y since:  2000-10-01 dict_keys(['AZR', 'T14', '600', 'D01', 'G07', 'LJ3', 'S20', 'H02', 'F13', 'BAZ'])
backtest 1Q 2001-10-01 2002-01-01 ByAdjClose:  0.244899 ByAddBackClose:  0.244330

Evaluate 1Y since:  2001-01-01 dict_keys(['T14', '600', 'AZR', 'G13', 'F13', 'D01', 'G07', 'BAZ', 'T15', 'G92'])
backtest 1Q 2002-01-01 2002-04-01 ByAdjClose:  0.089573 ByAddBackClose:  0.088380

Evaluate 1Y since:  2001-0

In [86]:
df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
returns = df['ByAddBackClose'] + 1
returns.product()

3.07166240833986

In [85]:
df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
returns = df['ByAdjClose'].dropna() + 1
returns.product()

6.466738845153519

# Rolling Test using Adj Close Return

In [ ]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAdjClose):
                returns[stock.ticker] = r.ByAdjClose

    # sort to find top 10 return stocks using Adj Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)
    if len(top_returns) == 10:
        r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
        print("backtest 1Q", period_end, eval_period_end, r)
        ByAdjClose.append(r.ByAdjClose)
        ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

# Manual portfolio

In [14]:
dy1 = ["M01", "5DD", "CC3", "i07", "BN4"]
dy2 = ["L38", "E5H", "1D0", "Z74", "CY6U"]
# ww = ["A04", "5VP", "BQD", "F03", "NO4", "41B", "A34", "T14", "Q0X"]

ww = ["ES3"]

ByAddBackClose = []
ByAdjClose = []


period_start = util.YearMonth(2014, 1)
period_last = util.YearMonth(2024, 1)

portfolio = stocks.loads(ww)
while period_start <= period_last:
    period_end = util.get_next(period_start, 3)

    # returns = stocks.get_returns(period_start, period_end, portfolio)
    # w1 = stocks.get_weights_by_returns(returns)

    r = stocks.get_portfolio_return(period_start, period_end, portfolio)
    print(period_start, period_end, r)
    ByAdjClose.append(r.ByAdjClose)
    ByAddBackClose.append(r.ByAddBackClose)

    # r = stocks.get_portfolio_return(period_end, backtest_period_end, portfolio, w1.ByAddBackClose)
    # print(backtest_period_start, backtest_period_end, r)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

get_overall_return(ByAdjClose, ByAddBackClose)

2014-01-01 2014-04-01 ByAdjClose:  0.004756 ByAddBackClose:  0.004037
2014-04-01 2014-07-01 ByAdjClose:  0.031348 ByAddBackClose:  0.031348
2014-07-01 2014-10-01 ByAdjClose:  0.019714 ByAddBackClose:  0.019817
2014-10-01 2015-01-01 ByAdjClose:  0.030303 ByAddBackClose:  0.030303
2015-01-01 2015-04-01 ByAdjClose:  0.020057 ByAddBackClose:  0.019941
2015-04-01 2015-07-01 ByAdjClose: -0.026163 ByAddBackClose: -0.026163
2015-07-01 2015-10-01 ByAdjClose: -0.141673 ByAddBackClose: -0.139763
2015-10-01 2016-01-01 ByAdjClose:  0.035087 ByAddBackClose:  0.035088
2016-01-01 2016-04-01 ByAdjClose:  0.005709 ByAddBackClose:  0.003793
2016-04-01 2016-07-01 ByAdjClose:  0.017668 ByAddBackClose:  0.017668
2016-07-01 2016-10-01 ByAdjClose:  0.021413 ByAddBackClose:  0.021453
2016-10-01 2017-01-01 ByAdjClose:  0.006849 ByAddBackClose:  0.006849
2017-01-01 2017-04-01 ByAdjClose:  0.096266 ByAddBackClose:  0.095608
2017-04-01 2017-07-01 ByAdjClose:  0.021875 ByAddBackClose:  0.021875
2017-07-01 2017-10-0

In [ ]:
# stock = stocks.load("D05")
# print(stock.get_sector())
# df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
# df.AvergeVolume = df.AvergeVolume / 10000
# df = df[df.AvergeVolume>1].sort_values(by=['AvergeVolume'])

# df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
# df.MarketCap = df.MarketCap / 1000000
# df = df[df.MarketCap>1000].sort_values(by=['MarketCap'])

# items = stocks.loads(df.Ticker.to_list())

period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2024, 1)

while period_start < period_last:
    period_end = util.get_next(period_start, 12)
    count = 0
    total_yield = 0
    for stock in items:
        df = stock.dividend
        dividend = df[df["Day"] > str(period_start)][df["Day"] < str(period_end)].Dividends.sum()
        sell_index = stock.price['Day'].searchsorted(str(period_end))
        if dividend > 0:
            dyield = dividend / stock.price.iloc[sell_index].Close
            count += 1
            total_yield += dyield
    print(period_start, ": ", count, " ", total_yield / count)
    period_start = period_end

# Divident Stats

In [4]:
stock_with_dividend = 0
for s in all_stocks:
    if len(s.dividend) > 0:
        stock_with_dividend +=1

print(len(all_stocks), " ", stock_with_dividend)

period_start = util.YearMonth(2015, 1)
period_last = util.YearMonth(2023, 1)

portfolio = stocks.loads(ww)
while period_start <= period_last:
    period_end = util.get_next(period_start, 12)

708   542


# Adj Close return vs AddBack Return

In [183]:
# stock = stocks.load("D05")
stock = stocks.load("D8DU")
df = stocks.get_addback_close(stock, "2022-01-01", "2024-01-01")

return_adj_close = df["Adj Close"].pct_change().dropna()
return_addback_close = df["AddBackClose"].pct_change().dropna()

print(return_addback_close.mean())
print(return_adj_close.mean())
return_adj_close.std()  / return_addback_close.std() - 1
# return_adj_close.mean()  / return_addback_close.mean() - 1

-0.0002463114195503293
-0.00044110314428133615


0.04505055363062538

In [195]:
df = pd.concat([return_addback_close, return_adj_close], axis=1)
df.cov().iloc[0, 1]

0.000718800466679042

# Stock Market Cap Distribution

In [ ]:
df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
df.MarketCap = df.MarketCap / 1000000
plt = df[df.MarketCap<100].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.legend(['Total 365 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M.png')

plt = df[df.MarketCap>100][df.MarketCap<1500].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 100M~1.5B SGD)')
plt.legend(['Total 188 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M_1.5B.png')

plt = df[df.MarketCap>1500][df.MarketCap<30000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 1.5B~30B SGD)')
plt.legend(['Total 62 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_1.5B_20B.png')

df[df.MarketCap>10000].sort_values(by=['MarketCap']).to_csv("mc_10B_plus.csv", index=False, float_format='%.6f')

# Average Daily Trading Volume Hist

In [ ]:
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000

plt = df[df.AvergeVolume<=1].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) - Below(10K SGD)')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 365 stocks'])
plt.get_figure().savefig('adtv_10K.png')

plt = df[1<df.AvergeVolume][df.AvergeVolume<=30].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 10K~300K SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 207 stocks'])
plt.get_figure().savefig('adtv_10K_300K.png')

plt = df[df.AvergeVolume>30][df.AvergeVolume<=300].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 300K~3M SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 68 stocks'])
plt.get_figure().savefig('adtv_300K_3M.png')

df[df.AvergeVolume>300].sort_values(by=['AvergeVolume']).to_csv("adtv_3M_plus.csv", index=False, float_format='%.6f')

# STI Estimate

In [ ]:
es3 = stocks.load("ES3")
df = stocks.get_addback_close(es3, "2010-03-01", "2024-03-15")
df["Adj Close"] = df["Adj Close"] * 990
df["AddBackClose"] = df["AddBackClose"] * 1010
df["Close"] = df["Close"] * 1000

plt = df.plot(x="Day", y = ["AddBackClose", "Close", "Adj Close"], linewidth=0.8, figsize=(15, 10), title="Esitmated STI")
plt.legend(['Estimated STI if add back dividend', 'STI', 'Estimated STI if use Adj Close'])
plt.get_figure().savefig('STI_estimated.png')